In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/405 Final Project"

[Errno 2] No such file or directory: '/content/drive/MyDrive/405 Final Project'
/content


In [ ]:
!ls -l "/content/drive/.shortcut-targets-by-id/16vC_ElnJuNcHxf_KgoFCBDED3h1xroyh/405 Final Project

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/16vC_ElnJuNcHxf_KgoFCBDED3h1xroyh/405 Final Project"

/content/drive/.shortcut-targets-by-id/16vC_ElnJuNcHxf_KgoFCBDED3h1xroyh/405 Final Project


In [ ]:
!spark-submit Revised_spark.py \
    "2017_citibike_morning_rush_07_09.csv,2018_citibike_morning_hours_07_09.csv,2018_citibike_rush_hours_15_20.csv,2019_citibike_morning_rush_07_09.csv" \
    "Air_Quality.csv" \
    "joined_output"

25/03/12 06:27:49 INFO SparkContext: Running Spark version 3.5.5
25/03/12 06:27:49 INFO SparkContext: OS info Linux, 6.1.85+, amd64
25/03/12 06:27:49 INFO SparkContext: Java version 11.0.26
25/03/12 06:27:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/12 06:27:49 INFO ResourceUtils: ==============================================================
25/03/12 06:27:49 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/12 06:27:49 INFO ResourceUtils: ==============================================================
25/03/12 06:27:49 INFO SparkContext: Submitted application: Revised_spark.py
25/03/12 06:27:50 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: c

In [ ]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_timestamp, date_trunc, row_number, split,
    regexp_extract, regexp_replace, when,
    monotonically_increasing_id, broadcast, date_format, to_date,
    avg, first
)
from pyspark.sql import Window

def main():
    """
    This script joins Citibike rides data with Air Quality data on matching date + zone.
    It falls back to default CSV/paths if you don't pass arguments.

    Usage (if providing arguments):
        spark-submit spark-job.py \
            [citibike_paths (comma-separated)] \
            [air_quality_path] \
            [output_path]
    """
    # If 3 user arguments are provided, parse them. Otherwise, use fallback defaults.
    if len(sys.argv) == 4:
        citibike_paths = sys.argv[1].split(",")
        air_quality_path = sys.argv[2]
        output_path = sys.argv[3]
    else:
        print("WARNING: No (or incomplete) arguments were provided.")
        print("Using default file paths. Adjust them as needed.\n")
        citibike_paths = [
            "2017_citibike_morning_rush_07_09.csv",
            "2018_citibike_morning_hours_07_09.csv",
            "2019_citibike_morning_rush_07_09.csv"
        ]
        air_quality_path = "Air_Quality.csv"
        output_path = "joined_output"

    # Create Spark session
    spark = SparkSession.builder.getOrCreate()

    ######################################################
    # Load and preprocess Citibike rides data
    ######################################################
    rides = (
        spark.read
            .option("header", "true")
            .option("recursiveFileLookup", "true")
            .option("spark.sql.files.maxPartitionBytes", "128MB")
            .csv(citibike_paths)
            .select(
                col("tripduration").alias("trip_duration"),
                col("starttime").alias("trip_start_time"),
                col("startstationid").alias("start_station_id"),
                col("startstationlatitude").alias("start_station_latitude"),
                col("startstationlongitude").alias("start_station_longitude"),
                col("endstationid").alias("end_station_id"),
                col("usertype").alias("user_type")
            )
    )

    # Determine borough based on starting station coordinates.
    rides = rides.withColumn(
        "zone",
        when(
            (col("start_station_latitude").between(40.70, 40.75)) &
            (col("start_station_longitude").between(-74.02, -73.93)),
            "lower-manhattan"
        ).when(
            (col("start_station_latitude").between(40.75, 40.82)) &
            (col("start_station_longitude").between(-74.02, -73.93)),
            "mid-manhattan"
        ).when(
            (col("start_station_latitude").between(40.82, 40.88)) &
            (col("start_station_longitude").between(-74.02, -73.93)),
            "upper-manhattan"
        ).when(
            (col("start_station_latitude").between(40.57, 40.70)) &
            (col("start_station_longitude").between(-74.05, -73.85)),
            "Brooklyn"
        ).when(
            (col("start_station_latitude").between(40.70, 40.80)) &
            (col("start_station_longitude").between(-73.93, -73.70)),
            "Queens"
        ).when(
            (col("start_station_latitude") >= 40.88) & (col("start_station_latitude") < 41.00) &
            (col("start_station_longitude").between(-73.93, -73.70)),
            "Bronx"
        ).otherwise("Unknown")
    )

    # Convert trip_start_time to timestamp for further processing.
    rides = rides.withColumn(
        "trip_start_timestamp",
        to_timestamp(col("trip_start_time"), "MM/dd/yyyy HH:mm:ss")
    )

    # -------------------------------------------------------
    # (REMOVED) the optional debugging write that caused errors:
    #
    # rides.write \
    #     .option("maxRecordsPerFile", 100000) \
    #     .mode("overwrite") \
    #     .parquet("../output/citibike_zones")
    # -------------------------------------------------------

    ######################################################
    # Load and preprocess Air Quality data
    ######################################################
    air_quality = (
        spark.read
            .option("header", "true")
            .option("recursiveFileLookup", "true")
            .option("spark.sql.files.maxPartitionBytes", "128MB")
            .csv(air_quality_path)
            .select(
                col("Indicator ID"),
                col("name").alias("pollutants"),
                col("measure"),
                col("Geo Type Name"),
                col("Geo Join ID"),
                col("Geo Place Name"),
                col("Start_Date")
            )
            # Convert "start_date" to a Spark date
            .withColumn("start_date", to_date(col("start_date"), "yyyy/MM/dd"))
            # Filter date between 2017 and 2019
            .filter(col("start_date").between("2017-01-01", "2019-12-31"))
            # Retain only records for UHF 42
            .filter(col("Geo Type Name") == "UHF 42")
    )

    # Map UHF zone numbers to broader district names
    air_quality = air_quality.withColumn(
        "zone",
        when(col("Geo Join ID").isin([308, 309, 310]), "lower-manhattan")
        .when(col("Geo Join ID").isin([306, 307]), "mid-manhattan")
        .when(col("Geo Join ID").isin([301, 302, 303, 304, 305]), "upper-manhattan")
        .when(col("Geo Join ID").isin([201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211]), "Brooklyn")
        .when(col("Geo Join ID").isin([101, 102, 103, 104, 105, 106, 107]), "Bronx")
        .when(col("Geo Join ID").isin([401, 402, 403, 404, 405, 406, 407, 408, 409, 410]), "Queens")
        .when(col("Geo Join ID").isin([501, 502, 503, 504]), "Staten Island")
        .otherwise("Unknown")
    )

    ######################################################
    # Join rides data with air quality data on zone + date
    ######################################################

    # 1) Create a date column for rides
    rides = rides.withColumn(
        "trip_start_date", date_format(col("trip_start_timestamp"), "yyyy-MM-dd")
    )

    # 2) Create a date column for Air Quality
    air_quality = air_quality.withColumn(
        "start_date_formatted", date_format(col("start_date"), "yyyy-MM-dd")
    )

    # 3) Join on (zone + date)
    joined_df = (
        rides.join(
            air_quality,
            (rides.zone == air_quality.zone) &
            (rides.trip_start_date == air_quality.start_date_formatted),
            "left"
        )
        .select(
            rides["*"],
            air_quality["pollutants"],
            air_quality["measure"],
            air_quality["Geo Join ID"],
            air_quality["Geo Place Name"]
        )
    )

    # 4) Aggregate measure to remove duplicates
    final = (
        joined_df
        .groupBy("trip_start_time", "trip_start_date", "zone", "pollutants")
        .agg(
            avg("measure").alias("measure"),
            first("Geo Join ID").alias("geo_join_id"),
            first("Geo Place Name").alias("geo_place_name")
        )
    )

    # Write the final joined data to parquet
    # Make sure 'output_path' points to a directory you can actually write to!
    final.write \
        .option("maxRecordsPerFile", 100000) \
        .mode("overwrite") \
        .parquet(output_path)

    spark.stop()


if __name__ == "__main__":
    main()


Using default file paths. Adjust them as needed.



AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/2017_citibike_morning_rush_07_09.csv.